In [1]:
import sys
sys.path.insert(1, '../src')
from imports import *
from dataset_utilities import *
from model_aplication import *
from hyperparameters_search import *

Preprocessing necessari per poder fer la execucciò

In [2]:
dataset = load_dataset('..\data\smart_grid_stability_augmented.csv')
label_encoder = sklearn.preprocessing.LabelEncoder()
dataset['stabf']= label_encoder.fit_transform(dataset['stabf'])
data = dataset.values
y = data[:,-2]
y_bin = data[:,-1].astype(int)
x = data[:, :12]

x_train, y_train, x_val, y_val = split_data(x, y_bin)
sel_ = SelectFromModel(LogisticRegression(C=1, penalty='l2', solver='liblinear'))
sel_.fit(x_train, np.ravel(y_train, order='C'))
sel_.get_support()
x_train = pd.DataFrame(x_train)
selected_feat = x_train.columns[(sel_.get_support())]
x_train_selected = sel_.transform(x_train)
x_test_selected = sel_.transform(x_val)
x_train_selected.shape, x_train_selected.shape
x_selected = np.concatenate((x_train_selected, x_test_selected))
y_selected = np.concatenate((y_train, y_val))


dataset_x = dataset.drop(columns='stabf')
scaler = MinMaxScaler()
data_rescaled = scaler.fit_transform(dataset_x)
pca2 = PCA(n_components = 2)
pca2.fit(data_rescaled)
reduced = pca2.transform(data_rescaled)
df2 = pd.DataFrame(reduced, columns = ['pc1','pc2'])
df2['stabf'] = dataset['stabf']
X2 = df2.drop(['stabf'],axis=1)
y2 = df2.filter(['stabf'])
pca95 = PCA(n_components = 0.95, svd_solver='full')
pca95.fit(data_rescaled)
reduced = pca95.transform(data_rescaled)
df95 = pd.DataFrame(reduced, columns = ['pc1','pc2','pc3','pc4','pc5','pc6','pc7','pc8','pc9','pc10','pc11'])
df95['stabf']=dataset['stabf']
X95 = df95.drop(['stabf'],axis=1)
y95 = df95.filter(['stabf'])

Fem una recerca d'hiperparametres per trobar els millors atributs del model classificador.

Recerca amb les millors classes del dataset

In [5]:
leaf_size = list(range(1,20))
n_neighbors = list(range(1,2))
p=[1,2]
n_splits=10
n_repeats=3
Hyperparam_search_KNC(x_selected,y_selected,n_splits,n_repeats,leaf_size,n_neighbors,p,"Grid search","../models/KnC_grid_little_nonPCA.sav")
Hyperparam_search_KNC(x_selected,y_selected,n_splits,n_repeats,leaf_size,n_neighbors,p,"Random search","../models/KnC_RS_little_nonPCA.sav")

Grid search
Best leaf_size: 1
Best p: 1
Best n_neighbors: 1
Precision at best_score: 0.6764444444444444
--- 21.749764919281006 seconds ---
Random search
Best leaf_size: 1
Best p: 1
Best n_neighbors: 1
Precision at best_score: 0.6772777777777778
--- 21.226409435272217 seconds ---


Recerca amb el PCA en 2 dimensions del dataset

In [6]:
leaf_size = list(range(1,20))
n_neighbors = list(range(1,2))
p=[1,2]
n_splits=10
n_repeats=3
Hyperparam_search_KNC(X2,y2,n_splits,n_repeats,leaf_size,n_neighbors,p,"Grid search","../models/KnC_grid_little.sav")
Hyperparam_search_KNC(X2,y2,n_splits,n_repeats,leaf_size,n_neighbors,p,"Random search","../models/KnC_RS_little.sav")


Grid search
Best leaf_size: 1
Best p: 2
Best n_neighbors: 1
Precision at best_score: 0.9993333333333333
--- 20.12352204322815 seconds ---
Random search
Best leaf_size: 1
Best p: 1
Best n_neighbors: 1
Precision at best_score: 1.0
--- 20.397759675979614 seconds ---


Amb més detall

In [5]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3)
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]
n_splits=10
n_repeats=3
Hyperparam_search_KNC(X2,y2,n_splits,n_repeats,leaf_size,n_neighbors,p,"Grid search","../models/KnC_grid.sav")
Hyperparam_search_KNC(X2,y2,n_splits,n_repeats,leaf_size,n_neighbors,p,"Random search","../models/KnC_RS.sav")


KeyboardInterrupt: 

Provem també amb el Random Forest amb totes les dades inicials.

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 300, num = 100)]
n_splits=5
n_repeats=3
max_depth =[int(x) for x in np.linspace(0, 3, 1)]
max_depth.append(None)
min_samples_split = [2]
min_samples_leaf = [1]
bootstrap = [False]


Hyperparam_search_RF(x_selected,y_selected,n_splits,n_repeats,n_estimators,max_depth,min_samples_split,min_samples_leaf,bootstrap,"Grid search","../models/R_f_little_grid.sav")
Hyperparam_search_RF(x_selected,y_selected,n_splits,n_repeats,n_estimators,max_depth,min_samples_split,min_samples_leaf,bootstrap,"Random search","../models/R_f_little_RD.sav")

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1100, num = 5)]
n_splits=10
n_repeats=3
max_depth =[int(x) for x in np.linspace(0, 10, 1)]
max_depth.append(None)
min_samples_split = [int(x) for x in np.linspace(0, 5, 1)]
min_samples_leaf = [int(x) for x in np.linspace(0, 5, 1)]
bootstrap = [True, False]
Hyperparam_search_RF(x_selected,y_selected,n_splits,n_repeats,n_estimators,cv,max_depth,min_samples_split,min_samples_leaf,bootstrap,"Grid search","../models/R_f_grid.sav")
Hyperparam_search_RF(x_selected,y_selected,n_splits,n_repeats,n_estimators,cv,max_depth,min_samples_split,min_samples_leaf,bootstrap,"Random search","../models/R_f_RD.sav")